Monte Carlo Convergence of Ridge Regression Conditions (24) and (25) vs. Trial Count M

In [1]:
import numpy as np
import torch
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
n = 200
d = 28 * 28
lam = 0.01

transform = transforms.ToTensor()
mnist_train = datasets.MNIST(root="./mnist_data", train=True, download=True, transform=transform)
all_data = mnist_train.data.numpy().astype(np.float64)
N_total = all_data.shape[0]
all_data = all_data.reshape(N_total, 784)

def run_monte_carlo(M):
    Sum_C2_n = np.zeros((d, d), dtype=np.float64)
    Sum_C3_n = np.zeros((d, d), dtype=np.float64)
    Sum_H_trace_n = 0.0
    Sum_C2_n1 = np.zeros((d, d), dtype=np.float64)
    Sum_C3_n1 = np.zeros((d, d), dtype=np.float64)
    Sum_H_trace_n1  = 0.0
    Sum_G_raw_n = np.zeros((d, d), dtype=np.float64)
    Sum_G_raw_n1 = np.zeros((d, d), dtype=np.float64)
    Sum_H_raw_n = 0.0
    Sum_H_raw_n1 = 0.0

    for trial in range(M):
        idx_n = np.random.choice(N_total, size=n,   replace=False)
        idx_n1 = np.random.choice(N_total, size=n+1, replace=False)

        Xn_raw = all_data[idx_n, :]
        Xn1_raw = all_data[idx_n1, :]

        mu_n, std_n = Xn_raw.mean(axis=0, keepdims=True), Xn_raw.std(axis=0, keepdims=True)
        std_n[std_n < 1e-8] = 1.0
        Xn = (Xn_raw - mu_n) / std_n

        mu_n1, std_n1 = Xn1_raw.mean(axis=0, keepdims=True), Xn1_raw.std(axis=0, keepdims=True)
        std_n1[std_n1 < 1e-8] = 1.0
        Xn1 = (Xn1_raw - mu_n1) / std_n1

        C_n = Xn.T @ Xn + lam * np.eye(d)
        C_n1  = Xn1.T @ Xn1 + lam * np.eye(d)

        inv_C_n  = np.linalg.inv(C_n)
        inv_C_n1 = np.linalg.inv(C_n1)

        C2_n  = inv_C_n @ inv_C_n
        C2_n1 = inv_C_n1 @ inv_C_n1

        C3_n  = C2_n @ inv_C_n
        C3_n1 = C2_n1 @ inv_C_n1

        Sum_C2_n    += C2_n
        Sum_C3_n    += C3_n
        Sum_G_raw_n += C2_n

        Sum_C2_n1     += C2_n1
        Sum_C3_n1     += C3_n1
        Sum_G_raw_n1  += C2_n1

        H_i_n  = np.linalg.norm(inv_C_n @ Xn.T, ord='fro')**2
        trace_n = np.trace(C3_n @ (Xn.T @ Xn))

        H_i_n1 = np.linalg.norm(inv_C_n1 @ Xn1.T, ord='fro')**2
        trace_n1 = np.trace(C3_n1 @ (Xn1.T @ Xn1))

        Sum_H_raw_n    += H_i_n
        Sum_H_trace_n  += trace_n
        Sum_H_raw_n1   += H_i_n1
        Sum_H_trace_n1 += trace_n1

    G_n   = (lam**2 / M) * Sum_G_raw_n
    G_n1  = (lam**2 / M) * Sum_G_raw_n1
    H_n   = (1.0 / M) * Sum_H_raw_n
    H_n1  = (1.0 / M) * Sum_H_raw_n1

    dG_n  = (2.0 * lam / M) * Sum_C2_n - (2.0 * lam**2 / M) * Sum_C3_n
    dH_n  = - (2.0 / M) * Sum_H_trace_n

    G_diff = G_n - G_n1
    min_eig_G_diff = np.min(np.linalg.eigvalsh(G_diff))

    if abs(dH_n) < 1e-12:
        return min_eig_G_diff, np.nan

    alpha = (H_n - H_n1) / dH_n
    M_star = G_diff - alpha * dG_n
    min_eig_Mstar = np.min(np.linalg.eigvalsh(M_star))

    return min_eig_G_diff, min_eig_Mstar

M_values = np.arange(1000, 10001, 1000)
eigs_24 = []
eigs_25 = []

print("Running experiments for multiple M...")
for M in M_values:
    print(f"  → M = {M}")
    eig24, eig25 = run_monte_carlo(M)
    eigs_24.append(eig24)
    eigs_25.append(eig25)

plt.figure(figsize=(10, 6))
plt.plot(M_values, eigs_24, marker='o', label='Condition (24): min eig(G_n - G_{n+1})')
plt.plot(M_values, eigs_25, marker='s', label='Condition (25): min eig(M*)')
plt.axhline(y=0, color='gray', linestyle='--')
plt.xlabel('Number of Monte Carlo Trials (M)')
plt.ylabel('Smallest Eigenvalue')
plt.title('Eigenvalue Behavior vs. Number of Monte Carlo Trials')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

Running experiments for multiple M...
  → M = 1000


KeyboardInterrupt: 

Monte Carlo Convergence of Ridge Regression Conditions (24) and (25)

In [ ]:
import numpy as np
import torch
from torchvision import datasets, transforms

n = 200
d = 28 * 28
M = 100
lam = 0.01

transform = transforms.ToTensor()
mnist_train = datasets.MNIST(root="./mnist_data", train=True, download=True, transform=transform)
N_total = all_data.shape[0]
all_data = all_data.reshape(N_total, 784)

Sum_C2_n = np.zeros((d, d), dtype=np.float64)
Sum_C3_n = np.zeros((d, d), dtype=np.float64)
Sum_H_trace_n = 0.0

Sum_C2_n1 = np.zeros((d, d), dtype=np.float64)
Sum_C3_n1 = np.zeros((d, d), dtype=np.float64)
Sum_H_trace_n1 = 0.0

Sum_G_raw_n = np.zeros((d, d), dtype=np.float64)
Sum_G_raw_n1 = np.zeros((d, d), dtype=np.float64)
Sum_H_raw_n = 0.0
Sum_H_raw_n1 = 0.0

print("Starting Monte Carlo (M = {}) …".format(M))

for trial in range(M):
    idx_n = np.random.choice(N_total, size=n,   replace=False)
    idx_n1 = np.random.choice(N_total, size=n+1, replace=False)

    Xn_raw = all_data[idx_n, :]
    Xn1_raw = all_data[idx_n1, :]

    mu_n = Xn_raw.mean(axis=0, keepdims=True)
    std_n = Xn_raw.std(axis=0, keepdims=True)

    std_n[std_n < 1e-8] = 1.0
    Xn = (Xn_raw - mu_n) / std_n

    mu_n1 = Xn1_raw.mean(axis=0, keepdims=True)
    std_n1 = Xn1_raw.std(axis=0, keepdims=True)
    std_n1[std_n1 < 1e-8] = 1.0
    Xn1 = (Xn1_raw - mu_n1) / std_n1

    C_n = Xn.T.dot(Xn)   + lam * np.eye(d)
    C_n1 = Xn1.T.dot(Xn1) + lam * np.eye(d)

    inv_C_n = np.linalg.inv(C_n)
    inv_C_n1 = np.linalg.inv(C_n1)

    C2_n = inv_C_n.dot(inv_C_n)
    C2_n1 = inv_C_n1.dot(inv_C_n1)

    C3_n = C2_n.dot(inv_C_n)
    C3_n1 = C2_n1.dot(inv_C_n1)

    Sum_C2_n += C2_n
    Sum_C3_n += C3_n
    Sum_G_raw_n += C2_n

    Sum_C2_n1 += C2_n1
    Sum_C3_n1 += C3_n1
    Sum_G_raw_n1 += C2_n1

    invC_n_XnT = inv_C_n.dot(Xn.T)
    H_i_n = np.linalg.norm(invC_n_XnT, ord='fro')**2
    Sum_H_raw_n += H_i_n

    trace_term_n = np.trace( C3_n.dot(Xn.T.dot(Xn)) )
    Sum_H_trace_n += trace_term_n

    invC_n1_Xn1T = inv_C_n1.dot(Xn1.T)
    H_i_n1 = np.linalg.norm(invC_n1_Xn1T, ord='fro')**2
    Sum_H_raw_n1 += H_i_n1

    trace_term_n1 = np.trace( C3_n1.dot(Xn1.T.dot(Xn1)) )
    Sum_H_trace_n1 += trace_term_n1

    if (trial+1) % 50 == 0:
        print(f"  → Completed {trial+1}/{M} trials")

print("Monte Carlo loop done.\n")

G_n = (lam**2 / M) * Sum_G_raw_n
G_n1 = (lam**2 / M) * Sum_G_raw_n1

H_n = (1.0 / M) * Sum_H_raw_n
H_n1 = (1.0 / M) * Sum_H_raw_n1

dG_n = (2.0*lam / M) * Sum_C2_n   -  (2.0 * lam**2 / M) * Sum_C3_n
dH_n = - (2.0 / M) * Sum_H_trace_n

dG_n1 = (2.0*lam / M) * Sum_C2_n1 -  (2.0 * lam**2 / M) * Sum_C3_n1
dH_n1 = - (2.0 / M) * Sum_H_trace_n1

G_diff = G_n - G_n1
eigvals_G_diff = np.linalg.eigvalsh(G_diff)
min_eig_G_diff = np.min(eigvals_G_diff)
H_diff = H_n - H_n1

alpha = H_diff / dH_n
M_star = G_diff - alpha * dG_n

eigvals_Mstar = np.linalg.eigvalsh(M_star)
min_eig_Mstar = np.min(eigvals_Mstar)

print("=== RESULTS ===")
print(f"Condition (24):  min_eig( G_n - G_{n+1} ) = {min_eig_G_diff:.6e}")
if min_eig_G_diff >= -1e-8:
    print("  → (24) appears satisfied (matrix is ≽ 0 up to numerical tol).")
else:
    print("  → (24) VIOLATED (smallest eigenvalue < 0).")
print()
print(f"Condition (25):  min_eig( (G_n - G_{n+1}) - ((H_n - H_{n+1})/(dH_n))·dG_n ) = {min_eig_Mstar:.6e}")
if min_eig_Mstar >= -1e-8:
    print("  → (25) appears satisfied (matrix is ≽ 0 up to numerical tol).")
else:
    print("  → (25) VIOLATED (smallest eigenvalue < 0).")


Starting Monte Carlo (M = 100) …
  → Completed 50/100 trials
  → Completed 100/100 trials
Monte Carlo loop done.

=== RESULTS ===
Condition (24):  min_eig( G_n – G_201 ) = -1.265244e-01
  → (24) VIOLATED (smallest eigenvalue < 0).

Condition (25):  min_eig( (G_n – G_201) – ((H_n – H_201)/(dH_n))·dG_n ) = -1.265225e-01
  → (25) VIOLATED (smallest eigenvalue < 0).

Done.
